# Sistemas de recomendación: Filtrado Colaborativo

Filtrado colaborativo utilizando Gradient Tape (Modelo Fino) para crear un sistema de recomendación de películas. 

## Esquema
- [ 1 - Contexto: Filtado Colaborativo](#1)
- [ 2 - Sistemas de recomendación](#2)
- [ 3 - Conjunto de datos de calificaciones de películas](#3)
- [ 4 - Algoritmo de aprendizaje de filtrado colaborativo](#4)
- [ 4.1 Función de costo de filtrado colaborativo](#4.1)
- [ Implementación Algoritmo](#ex01)
- [ 5 - Aprendizaje de recomendaciones de películas](#5)
- [ 6 - Recomendaciones](#6)



##  Paquetes

In [1]:
# Librerias, Visualizacion, Procesamiento de datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Tensorflow, Red Neuronal
import tensorflow as tf
from tensorflow import keras

pd.set_option("display.precision", 1)
pd.set_option('display.max_columns', None)

# Libreria Personalizada: Leer, Normalizar y Procesar set de datos.
from recsys_utils import *

import warnings
warnings.filterwarnings('ignore')


## 1. Contexto: Filtado Colaborativo


## Notación del Modelo de Predicción de Calificaciones

| **Notación<br />General** | **Descripción** | **Python** |
|---|---|---|
| $r(i,j)$ | Escalar; = 1 si el usuario j calificó la película i, = 0 en caso contrario | `R[i, j]` |
| $y(i,j)$ | Escalar; = Calificación dada por el usuario j a la película i (si $r(i,j) = 1$ está definido) | `Y[i, j]` |
| $\mathbf{w}^{(j)}$ | Vector; Parámetros para el usuario j | `W[j]` |
| $b^{(j)}$ | Escalar; Parámetro de sesgo para el usuario j | `b[j]` |
| $\mathbf{x}^{(i)}$ | Vector; Calificaciones de características para la película i | `X[i]` |
| $n_u$ | Número de usuarios | `num_users` |
| $n_m$ | Número de películas | `num_movies` |
| $n$ | Número de características | `num_features` |
| $\mathbf{X}$ | Matriz de vectores $\mathbf{x}^{(i)}$ | `X` |
| $\mathbf{W}$ | Matriz de vectores $\mathbf{w}^{(j)}$ | `W` |
| $\mathbf{b}$ | Vector de parámetros de sesgo $b^{(j)}$ | `b` |
| $\mathbf{R}$ | Matriz de elementos $r(i,j)$ | `R` |

**Explicación:**

* La notación $r(i,j)$ indica si un usuario específico ha calificado una película específica.
* $y(i,j)$ representa la calificación real de la película dada por el usuario.
* $\mathbf{w}^{(j)}$ y $b^{(j)}$ representan los parámetros del modelo para cada usuario, que se utilizan para predecir las calificaciones.
* $\mathbf{x}^{(i)}$ representa las características de una película específica.
* Las variables `num_users`, `num_movies` y `num_features` representan las dimensiones del conjunto de datos.
* `X`, `W`, `b` y `R` son las matrices que representan los datos en Python. 

**Ejemplo:**

Si tenemos 5 usuarios y 3 películas, las matrices `r` (Usuario Calificó la pelicula, si=1, no=0) y `j` (calificacion dada por el Usario si R=1) podrían verse así:

## Resumen de Calificaciones:

| Usuario r(i,j) | Película 1 | Película 2 | Película 3 | Calificación Real y(i,j)|
|---|---|---|---|---|
| User 1 | 1 | 0 | 1 | [4, 0, 5] |
| User 2 | 0 | 1 | 0 | [0, 3, 0] |
| User 3 | 1 | 0 | 1 | [2, 0, 3] |
| User 4 | 0 | 1 | 1 | [0, 4, 2] |
| User 5 | 1 | 1 | 0 | [5, 1, 0] |

**Explicación:**

* La columna "Película" indica si el usuario ha calificado la película (Sí=1/No=0).
* La columna "Calificación Real" muestra las calificaciones reales proporcionadas por cada usuario para las películas que han calificado.

**Ejemplo:**

* El usuario 1 ha calificado las películas 1 y 3 con una calificación de 4 y 5, respectivamente.
* El usuario 2 solo ha calificado la película 2 con una calificación de 3.
* El usuario 3 ha calificado las películas 1 y 3 con una calificación de 2 y 3, respectivamente.

**Nota:**

Ejemplo para visualizar y entender qué usuarios han calificado qué películas y sus calificaciones correspondientes.





## 2. Sistemas de recomendación

Los sistemas de recomendación son un conjunto de algoritmos diseñados para proporcionar recomendaciones a usuarios basándose en sus preferencias y historial de compras. Hay varios tipos de sistemas de recomendación, como:

- Filtrado Colaborativo: Este tipo de sistema utiliza la información de las preferencias de todos los usuarios para predecir las preferencias de un usuario específico.
- Filtrado Basado en Contenido: Este tipo de sistema utiliza la información de características de los items para predecir las preferencias de un usuario.
- Sistemas de Recomendación Basados en Redes: Este tipo de sistema utiliza la información de las preferencias de los usuarios y las relaciones entre los usuarios y los items para predecir las preferencias de un usuario.

Los sistemas de recomendación de filtrado colaborativo son una técnica popular para generar recomendaciones basadas en la información de las preferencias de los usuarios. En esta sección, veremos cómo implementar un sistema de recomendación de filtrado colaborativo utilizando Python y TensorFlow.

El objetivo de un sistema de recomendación de filtrado colaborativo es generar dos vectores: para cada usuario, un "vector de parámetros" que represente los gustos cinematográficos de un usuario. Para cada película, un vector de características del mismo tamaño que represente alguna descripción de la película. El producto escalar de los dos vectores más el término de sesgo debería producir una estimación de la clasificación que el usuario podría otorgar a esa película.

El diagrama a continuación detalla cómo se aprenden estos vectores:

<figure>
   <img src="./images/ColabFilterLearn.PNG"  style="width:740px;height:250px;" >
</figure>

Las calificaciones existentes se proporcionan en forma de matriz como se muestra. $Y$ contiene calificaciones; 0,5 a 5 inclusive en pasos de 0,5. 0 si la película no ha sido calificada. $R$ tiene un 1 si las películas han sido calificadas. Las películas están en filas, los usuarios en columnas. Cada usuario tiene un vector de parámetros $w^{user}$ y sesgo. Cada película tiene un vector de características $x^{movie}$. Estos vectores se aprenden simultáneamente utilizando las calificaciones de usuario/película existentes como datos de entrenamiento. Un ejemplo de entrenamiento se muestra arriba: $\mathbf{w}^{(1)} \cdot \mathbf{x}^{(1)} + b^{(1)} = 4$. Vale la pena señalar que el vector de características $x^{movie}$ debe satisfacer a todos los usuarios, mientras que el vector de usuario $w^{user}$ debe satisfacer a todas las películas. Este es el origen del nombre de este enfoque: todos los usuarios colaboran para generar el conjunto de calificaciones.

<figure>
   <img src="./images/ColabFilterUse.PNG"  style="width:640px;height:250px;" >
</figure>

Una vez que se aprenden los vectores de características y los parámetros, se pueden utilizar para predecir cómo calificaría un usuario una película sin calificación. Esto se muestra en el diagrama anterior. La ecuación es un ejemplo de predicción de una calificación para el usuario uno en una película cero.

**Objetivos Importantes (Por estructurar):**
1. Implementar función para calcular objetivo del filtrado colaborativo (Funcion de Costo).
2. Esta función mide qué tan bien el modelo de filtrado colaborativo predice las calificaciones de los usuarios. Cuanto menor sea el valor que        devuelva la función, mejor será el desempeño del modelo.
3. Entrenamiento del Modelo: Después de implementar la Funcion, usaremos un bucle de entrenamiento de TensorFlow (Gradient Tape) para ajustar los parámetros del modelo de filtrado colaborativo. Este bucle iterativamente ajustará los parámetros del modelo para minimizar la función objetivo calculada por la funcion.
4. Conjunto de Datos y Estructuras de Datos: Antes de implementar la función objetivo y el entrenamiento, hay que definir las estructuras de datos que se utilizarán en el Poryecto asi como tambien entender la problematica planteada como; matrices que representan las calificaciones de los usuarios, información sobre las películas o incluso características de los usuarios.

## Ejemplo del Filtrado Colaborativo con 3 Usuarios y 4 Películas

Supongamos que tenemos 3 usuarios (A, B, C) y 4 películas (1, 2, 3, 4). 

**Matriz de Calificaciones (Y):**

| Usuario | Película 1 | Película 2 | Película 3 | Película 4 |
|---|---|---|---|---|
| A | 5 | 3 | 0 | 4 |
| B | 2 | 4 | 5 | 1 |
| C | 0 | 1 | 3 | 0 |

**Matriz de Indicadores (R):**

| Usuario | Película 1 | Película 2 | Película 3 | Película 4 |
|---|---|---|---|---|
| A | 1 | 1 | 0 | 1 |
| B | 1 | 1 | 1 | 1 |
| C | 0 | 1 | 1 | 0 |

**El objetivo del filtrado colaborativo es aprender:**

* **Vectores de parámetros para cada usuario (Modo de Ejemplo):**
    * `w^(A) = [1, 0.5, 0.2, 0.8]` 
    * `w^(B) = [0.5, 1, 0.8, 0.2]` 
    * `w^(C) = [0.2, 0.8, 0.6, 0.1]` 

* **Vectores de características para cada película (Modo de Ejemplo):**
    * `x^(1) = [1, 0, 0, 0]`
    * `x^(2) = [0, 1, 0, 0]`
    * `x^(3) = [0, 0, 1, 0]`
    * `x^(4) = [0, 0, 0, 1]`

**Ahora queremos predecir la calificación que el usuario C le daría a la película 1, usando la ecuación `y = w · x + b`:**

1. **Obtener los vectores:**
    * `w^(C) = [0.2, 0.8, 0.6, 0.1]` (vector de parámetros del usuario C)
    * `x^(1) = [1, 0, 0, 0]` (vector de características de la película 1)
    * `b^(C) = 0.3` (sesgo del usuario C)

**Nota: El sesgo b^(C) juega un papel crucial en el filtrado colaborativo al corregir las tendencias generales de calificación que existen entre las películas. Algunas películas tienden a recibir calificaciones más altas que otras, independientemente del usuario. El modelo ajusta el sesgo b^(C) para compensar este desequilibrio, buscando el mejor equilibrio entre las predicciones y las calificaciones reales. Al minimizar el error, el modelo optimiza su rendimiento y proporciona recomendaciones más precisas.**

2. **Calcular el producto escalar:** 
    * `w^(C) · x^(1) = (0.2 * 1) + (0.8 * 0) + (0.6 * 0) + (0.1 * 0) = 0.2`

3. **Sumar el sesgo:**
    * `0.2 + 0.3 = 0.5`

4. **Predicción:**
    * La predicción de la calificación que el usuario C le daría a la película 1 es 0.5.

**Explicación:**

* El modelo ha aprendido que el usuario C tiene una preferencia mayor por las películas con las características 2 y 3.
* La película 1 tiene una alta puntuación en la característica 1 y una baja puntuación en las características 2 y 3. 
* Por lo tanto, la predicción es moderada, reflejando que el usuario C probablemente no disfrutaría mucho de la película 1.

**Otras Indicaciones:**
- Preferencias del usuario C: El vector de parámetros del usuario C (w^(C) = [0.2, 0.8, 0.6, 0.1]) indica que el usuario tiene una preferencia más alta por las películas con las características 2 y 3 (debido a que los valores en esas posiciones son más altos: 0.8 y 0.6).
- Características de la película 1: El vector de características de la película 1 (x^(1) = [1, 0, 0, 0]) indica que la película 1 tiene una alta puntuación en la característica 1 y una baja puntuación en las características 2 y 3.
- Predicción moderada: Como la película 1 tiene una puntuación baja en las características que el usuario C prefiere (características 2 y 3), la predicción del modelo es moderada. Esto significa que el modelo estima que el usuario C probablemente no disfrutaría mucho de la película 1.

**En resumen:** 

* El modelo ha aprendido vectores de parámetros para cada usuario y vectores de características para cada película. 
* Estos vectores representan las preferencias de los usuarios y las características de las películas. 
* Se utiliza la ecuación `y = w · x + b` para predecir la calificación que un usuario podría dar a una película que no ha visto.

**Nota:** Los valores de los vectores de parámetros y características son ejemplos. En realidad, estos valores se aprenderían durante el entrenamiento del modelo utilizando técnicas de aprendizaje automático. 


## 3. Conjunto de Datos de Calificaciones de Películas 🎬

Este proyecto explorará el filtrado colaborativo, una técnica de recomendación que se basa en las preferencias de otros usuarios para predecir las preferencias de un usuario dado, utilizando un conjunto de datos de calificaciones de películas derivado de [MovieLens "ml-latest-small"](https://grouplens.org/datasets/movielens/latest/), un conjunto de datos ampliamente utilizado en el campo de la recomendación.  

**El conjunto de datos original contiene 9000 películas calificadas por 600 usuarios.** Para simplificar el Entrenamiento se redujo el tamaño del conjunto de datos a películas lanzadas después del año 2000.  Este conjunto de datos reducido contiene **443 usuarios ($n_u$) y 4778 películas ($n_m$)**.

**Las calificaciones se basan en una escala de 0.5 a 5, con incrementos de 0.5.**

**Estructura de los Datos:**

* **Matriz Y:** 
    *  Es una matriz de $n_m \times n_u$ que almacena las calificaciones $y^{(i,j)}$, donde:
        * `i` representa el índice de la película.
        * `j` representa el índice del usuario.
    * Si el usuario `j` ha calificado la película `i`, la celda correspondiente en la matriz `Y` contiene la calificación. De lo contrario, contiene 0. 
    * **Ejemplo:**
    ```
    Y = [
        [5, 3, 0, 4],  # Película 1
        [2, 4, 5, 1],  # Película 2
        [0, 1, 3, 0],  # Película 3
        ...
    ]
    ```
    
* **Matriz R:**
    * Es una matriz de indicadores binarios de $n_m \times n_u$ que indica si un usuario ha calificado una película o no.
    * `R(i,j) = 1` si el usuario `j` ha calificado la película `i`, y `R(i,j) = 0` en caso contrario.
    * **Ejemplo:**
    ```
    R = [
        [1, 1, 0, 1],  # Película 1
        [1, 1, 1, 1],  # Película 2
        [0, 1, 1, 0],  # Película 3
        ...
    ]
    ```

* **Matrices X, W y b:**
    * `X` es una matriz de $n_m \times 10$ que contiene los vectores de características `x^(i)` de las películas, donde `i` es el índice de la película. Estos vectores representan las características de cada película, como el género, el director, los actores, etc.
    * `W` es una matriz de $n_u \times 10$ que contiene los vectores de parámetros `w^(j)` de los usuarios, donde `j` es el índice del usuario. Estos vectores representan las preferencias de cada usuario, como el gusto por diferentes géneros o actores.
    * `b` es un vector de $n_u$ elementos que contiene los sesgos `b^(j)` de los usuarios. El sesgo es un valor que representa la tendencia general de un usuario a dar calificaciones más altas o más bajas.

    **Ejemplo:** (Para 3 películas y 2 usuarios, con vectores de 3 dimensiones):

    ```
    X = 
      [
        [x^(0)_1, x^(0)_2, x^(0)_3],  # Vector de características de la película 0
        [x^(1)_1, x^(1)_2, x^(1)_3],  # Vector de características de la película 1
        [x^(2)_1, x^(2)_2, x^(2)_3]   # Vector de características de la película 2
      ] 
    ```

    ```
    W = 
      [
        [w^(0)_1, w^(0)_2, w^(0)_3],  # Vector de parámetros del usuario 0
        [w^(1)_1, w^(1)_2, w^(1)_3]   # Vector de parámetros del usuario 1
      ]
    ```

    ```
    b = 
      [
        b^(0),  # Sesgo del usuario 0
        b^(1)   # Sesgo del usuario 1
      ]
    ```

**En este Proyecto:**

* Cargaremos las matrices `Y` y `R` con el conjunto de datos de calificaciones de películas.
* Cargaremos las matrices `X`, `W` y `b` con valores precalculados. Estos valores se aprenderán más adelante porel Modelo, pero usaremos valores precalculados para desarrollar el modelo de costo. 

**Entender la estructura de los datos es esencial para construir el modelo de filtrado colaborativo y comprender el funcionamiento del sistema de recomendación.**

A lo largo de este proyecto, también se trabajarás con
matrices, $\mathbf{X}$, $\mathbf{W}$ and $\mathbf{b}$: 

$$\mathbf{X} = 
\begin{bmatrix}
--- (\mathbf{x}^{(0)})^T --- \\
--- (\mathbf{x}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{x}^{(n_m-1)})^T --- \\
\end{bmatrix} , \quad
\mathbf{W} = 
\begin{bmatrix}
--- (\mathbf{w}^{(0)})^T --- \\
--- (\mathbf{w}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{w}^{(n_u-1)})^T --- \\
\end{bmatrix},\quad
\mathbf{ b} = 
\begin{bmatrix}
 b^{(0)}  \\
 b^{(1)} \\
\vdots \\
b^{(n_u-1)} \\
\end{bmatrix}\quad
$$ 

In [2]:
#Cargar datos de la Libraria Personalizada.
X, W, b, num_movies, num_features, num_users = load_precalc_params_small() 
Y, R = load_ratings_small()

print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

Y (4778, 443) R (4778, 443)
X (4778, 10)
W (443, 10)
b (1, 443)
num_features 10
num_movies 4778
num_users 443


In [3]:
# A partir de la matriz, podemos calcular estadísticas como la calificación promedio.
tsmean =  np.mean(Y[0, R[0, :].astype(bool)])
print(f"Average rating for movie 1 : {tsmean:0.3f} / 5" )

Average rating for movie 1 : 3.400 / 5


## 4. Algoritmo de aprendizaje de filtrado colaborativo


- Definir una función objetivo que mida el error de predicción del modelo.
- El algoritmo debe ajustar los vectores de parámetros y características para minimizar la función objetivo, lo que mejora la precisión de las predicciones.

El algoritmo de filtrado colaborativo en el contexto de recomendaciones de películas considera un conjunto de vectores de parámetros de $n$ dimensiones
$\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)}$, $\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$ y $b^{(0)},...,b^{(n_u-1)}$, donde el
modelo predice la calificación de la película $i$ por el usuario $j$ como
$y^{(i,j)} = \mathbf{w}^{(j)}\cdot \mathbf{x}^{(i)} + b^{(j)}$ . Dado un conjunto de datos que consiste en
un conjunto de calificaciones producidas por algunos usuarios sobre algunas películas, se desea
conocer los vectores de parámetros $\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},
\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$ y $b^{(0)},...,b^{(n_u-1)}$ que producen el mejor ajuste (minimizan
el error al cuadrado).


### 4.1 Función de costo de filtrado colaborativo

La función de costo de filtrado colaborativo está dada por
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \left[ \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+ \underbrace{\left[
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
\right]}_{regularization}
\tag{1}$$
La primera suma en (1) es "para todo $i$, $j$ donde $r(i,j)$ es igual a $1$" y podría escribirse:

$$
= \left[ \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+\text{regularización}
$$

la funcion 'cofiCostFunc' para este caso (función de costo de filtrado colaborativo) deve devolver el costo (Cost).

In [6]:
# Prueva: Funcion de Costo No Vectorizada

def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Devuelve el costo del filtrado basado en contenido
      Args:
      X (ndarray (num_movies,num_features)): matriz de características de los elementos
      W (ndarray (num_users,num_features)) : matriz de parámetros de usuario
      b (ndarray (1, num_users) : vector de parámetros de usuario
      Y (ndarray (num_movies,num_users) : matriz de calificaciones de películas por parte de los usuarios
      R (ndarray (num_movies,num_users) : matriz, donde R(i, j) = 1 si la i-ésima película fue calificada por el j-ésimo usuario
      lambda_ (float): parámetro de regularización
    Devuelve:
      J (float) : costo
    """
    nm, nu = Y.shape
    J = 0
    
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    J = J/2

    # Regularización
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))

    return J

In [7]:
# Reducir el tamaño del conjunto de datos para que esto se ejecute más rápido

num_users_r = 4
num_movies_r = 5 
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Prueva Evaluar cost function no Vectorizada
#J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0);
#print(f"Cost: {J:0.2f}")
# Cost: 13.67

Cost: 13.67


In [8]:
# Prueva Evaluar, cost function Vectorizada con regularización
 
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

# Cost (with regularization): 28.09

Cost (with regularization): 28.09


**Implementación vectorizada**

Nota: Es importante crear una implementación vectorizada para calcular $J$, ya que luego se la llamará muchas veces durante la optimización.

In [10]:
# Funcion de Costo Vectorizada
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
  
    """
    Devuelve el costo del filtrado basado en contenido
    Vectorizado para mayor velocidad. Utiliza operaciones de tensorflow para ser compatible con el bucle de entrenamiento personalizado.
    Argumentos:
      X (ndarray (num_movies,num_features)): matriz de características de los elementos
      W (ndarray (num_users,num_features)) : matriz de parámetros de usuario
      b (ndarray (1, num_users) : vector de parámetros de usuario
      Y (ndarray (num_movies,num_users) : matriz de calificaciones de películas por parte de los usuarios
      R (ndarray (num_movies,num_users) : matriz, donde R(i, j) = 1 si la i-ésima película fue calificada por el j-ésimo usuario
      lambda_ (float): parámetro de regularización
      Devuelve:
      J (float) : costo
    """
    # Reformular variables para el broadcasting
    #X = tf.reshape(X, [-1, 1])
    #W = tf.reshape(W, [-1, 1])
    #b = tf.reshape(b, [-1])
    #Y = tf.reshape(Y, [-1])
    #R = tf.reshape(R, [-1])

   # Calcular el costo:
   # El broadcastingse se utiliza para realizar operaciones elemento por elemento en tensores de diferentes formas
   # En este caso, la utilizamos para multiplicar los elementos correspondientes de X, W e Y, y luego multiplicamos por R
   # El tensor resultante tiene forma (num_movies*num_users,)
   # La suma sobre este tensor nos da la suma de las diferencias al cuadrado entre las calificaciones predichas y reales para cada par usuario-película
   # Finalmente, dividimos por 2 y agregamos el término de regularización para obtener el costo J
    
    # Calcular el costo sin Regularización
    # j = (np.dot(X, W.T) + b - Y)*R
    # J = 0.5 * np.sum(j**2)
    
    # Calcular el costo Con Regularización
    # j = (tf.matmul(X, tf.transpose(W)) + b - Y)*R
    # J = 0.5 * tf.reduce_sum(j**2)
    
    # Añadir Regularización al coste
    # j += (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    
    # Devuelve el coste J como un tensor escalar
    # J = tf.squeeze(J) # Eliminar dimensiones singleton si están presentes (si J es un escalar)
  
    # j = tf.reduce_sum((tf.matmul(X, tf.transpose(W)) + b - Y)*R)
  
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))

    return J

In [11]:
# Evaluar la función de costo
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

# Evaluar la función de costo con regularización
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost: 13.67
Cost (with regularization): 28.09


## 5. Aprender recomendaciones de películas

Ahora que tenemos la función de costo vectorizada y un conjunto de datos reducido, se puede implementar el algoritmo de aprendizaje de recomendaciones colaborativo (entrenar algoritmo para que haga recomendaciones de películas.)


Nota: Puede ingresar sus propias opciones de películas (Valores Numericos). El algoritmo hará recomendaciones, si gustas, tambien puede cambiar para que coincida con tus gustos.

## Cargar las películas que te gustan

En el archivo [movie list](data/small_movie_list.csv) hay una lista de todas las películas del conjunto de datos.

In [12]:
movieList, movieList_df = load_Movie_List_pd()

my_ratings = np.zeros(num_movies) # Inicializar calificaciones (Matrices del Tamaño de num_movies nxm)

# Consulte el archivo small_movie_list.csv para conocer el ID de cada película del conjunto de datos
# Por ejemplo, Toy Story 3 (2010) tiene el ID 2700, por lo que para calificarla con "5", puede establecer
my_ratings[2700] = 5 

#O supongamos que no te gustó Persuasion (2007), puedes configurar
my_ratings[2609] = 2;

# Seleccionando algunas películas que nos gustaron y tambien las que no nos gustaron y las calificaciones que les hemos dado son las siguientes:
# Recordar Ver el archivo: data/small_movie_list.csv para buscar las Peliculas...


#my_ratings[1953] = 4   # Toy Story 3 (2010)
#my_ratings[125]  = 4   # Star Wars: Episode IV - A New Hope (1977)
#my_ratings[318]  = 5   # Inception (2010)
#my_ratings[1193] = 4   # Pulp Fiction (1994)
#my_ratings[50]   = 4   # Forrest Gump (1994)

my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)

# Filtrar películas que se calificaron y que no están en la lista de películas
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}');


# Filtrar películas que no se calificaron y que están en la lista de películas
#not_rated = [i for i in range(len(my_ratings)) if my_ratings[i] == 0 and i in movieList_df.index]

#print('\nNot rated movies:')
#for i in not_rated:
#    print(f'{movieList_df.loc[i,"title"]}')



New user ratings:

Rated 5.0 for  Shrek (2001)
Rated 5.0 for  Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Rated 2.0 for  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Rated 5.0 for  Harry Potter and the Chamber of Secrets (2002)
Rated 5.0 for  Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Rated 5.0 for  Lord of the Rings: The Return of the King, The (2003)
Rated 3.0 for  Eternal Sunshine of the Spotless Mind (2004)
Rated 5.0 for  Incredibles, The (2004)
Rated 2.0 for  Persuasion (2007)
Rated 5.0 for  Toy Story 3 (2010)
Rated 3.0 for  Inception (2010)
Rated 1.0 for  Louis Theroux: Law & Disorder (2008)
Rated 1.0 for  Nothing to Declare (Rien à déclarer) (2010)


Nota: Ahora, agreguegar estas reseñas a $Y$ y $R$ y normalicemos las calificaciones.

In [13]:
# Reload ratings
Y, R = load_ratings_small()

# Agregar nuevas calificaciones de usuarios a Y (Calificciones)
Y = np.c_[my_ratings, Y]

# Add new user indicator matrix to R (Indicadores)
R = np.c_[(my_ratings != 0).astype(int), R]

# Normalizar el set de datos
Ynorm, Ymean = normalizeRatings(Y, R) # Funcion Personalizada (rescsys_utils.py)

- Entrenar Modelo. 
- Inicializar Parámetros, 
- Seleccionar optimizador Adam (SGD, Adadelta, RMSpropAdagrap, etc).

# Valores útiles

num_movies, num_users = Y.shape
num_features = 100

# Establecer parámetros iniciales (W, X), use tf.Variable para rastrear estas variables

tf.random.set_seed(1234) # datos Irreproducibles (resultados consistentes)

W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.

optimizer = tf.optimizers.Adam(learning_rate=1e-1)
Now, let's write the training loop. We'll use the Adam optimizer and the mean squared error (MSE) loss function.

def train_step(inputs):
    Y, R = inputs
    with tf.GradientTape() as tape:
    
    # Compute the predicted ratings
    predictions = tf.matmul(W, X) + b
    # Compute the mean squared error loss
    loss = tf.reduce_sum((predictions - Y) ** 2) / tf.reduce_sum(R)
    return loss
    

In [14]:
# Valores útiles
num_movies, num_users = Y.shape
num_features = 100

# Establecer parámetros iniciales (W, X), use tf.Variable para rastrear estas variables
tf.random.set_seed(1234) # datos Irreproducibles (resultados consistentes)

W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Optimizedor
optimizer = keras.optimizers.Adam(learning_rate=1e-1) # lr=0.001, 0.01, 0.1

Let's now train the collaborative filtering model. This will learn the parameters $\mathbf{X}$, $\mathbf{W}$, and $\mathbf{b}$. 

## Descenso de Gradiente: Ajuste del Modelo de Filtrado Colaborativo

El algoritmo de filtrado colaborativo aprende los parámetros del modelo (los vectores de características `x`, los vectores de parámetros `w` y los sesgos `b`) mediante un proceso llamado **descenso de gradiente**. Este proceso busca minimizar el error entre las calificaciones predichas y las reales.

**Pasos del Descenso de Gradiente:**

1. **Inicialización:** Se inician valores aleatorios para los parámetros `x`, `w` y `b`.
2. **Bucle de Entrenamiento:** Se repite el siguiente proceso hasta que el modelo alcanza un nivel de precisión deseado (convergencia).
    * **Paso hacia adelante (forward pass):**
        * Se calcula la predicción de la calificación para cada usuario y película utilizando la ecuación `y^(i,j) = w^(j) · x^(i) + b^(j)`.
    * **Cálculo de la pérdida:** 
        * Se calcula la pérdida, que es una medida de la diferencia entre las predicciones y las calificaciones reales. 
        * La pérdida más común es el error cuadrático medio (MSE, RMSE, MAE, etc).
    * **Cálculo de las derivadas:**
        * Se calculan las derivadas de la pérdida respecto a cada parámetro (x, w, b). Esto indica la dirección en que cada parámetro debe cambiar para minimizar la pérdida.
    * **Actualización de parámetros:**
        * Los parámetros se actualizan en la dirección opuesta al gradiente (la dirección de la pendiente más pronunciada de la función de pérdida) utilizando una tasa de aprendizaje. La tasa de aprendizaje determina cuánto se ajustan los parámetros en cada paso. 
3. **Convergencia:** El proceso de entrenamiento se detiene cuando la pérdida alcanza un nivel bajo o deja de disminuir significativamente.

**Bucle de Entrenamiento Personalizado:**

El proceso de descenso de gradiente se implementa a través de un bucle de entrenamiento personalizado. Este bucle controla el flujo de los pasos mencionados anteriormente, iterando hasta que se alcanza la convergencia.

**Ejemplo de Código (TensorFlow):**

```python
import tensorflow as tf

# Definir la función de pérdida
def loss_function(x, w, b, Y, R):
    # Calcular las predicciones
    predictions = tf.matmul(W, tf.transpose(X)) + b
    # Calcular la pérdida (error cuadrático medio)
    loss = tf.reduce_mean(tf.square((Y - predictions) * R))
    return loss

# Crear un optimizador (por ejemplo, Adam)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Bucle de entrenamiento
epochs = 100
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        # Calcular la pérdida
        loss = loss_function(X, W, b, Y, R)

    # Calcular los gradientes
    gradients = tape.gradient(loss, [X, W, b])

    # Actualizar los parámetros
    optimizer.apply_gradients(zip(gradients, [X, W, b]))

    # Imprimir la pérdida (opcional)
    print(f"Época {epoch + 1}: Pérdida = {loss.numpy()}")
```

**TensorFlow para el Cálculo de Derivadas:**

TensorFlow proporciona herramientas para calcular derivadas de forma eficiente. La función `tf.GradientTape()` registra las operaciones en las variables de TensorFlow durante un "paso hacia adelante" del cálculo. Posteriormente, `tape.gradient()` calcula los gradientes de la pérdida respecto a las variables rastreadas. Los gradientes se pueden aplicar a los parámetros utilizando un optimizador.

**Nota:** El descenso de gradiente es un proceso iterativo que ajusta los parámetros del modelo para minimizar la pérdida, mejorando así la precisión de las predicciones. El bucle de entrenamiento personalizado controla este proceso, y TensorFlow facilita el cálculo de los gradientes necesarios para ajustar los parámetros.

    


## Pasos del Descenso de Gradiente, Formulación Matemática

**Inicialización:**

* Se inicializan valores aleatorios para los parámetros:
    * $x^{(i)}$ (vector de características de la película $i$)
    * $w^{(j)}$ (vector de parámetros del usuario $j$)
    * $b^{(j)}$ (sesgo del usuario $j$)

**Bucle de Entrenamiento:**

* Se repite hasta la convergencia:
    * **Paso hacia adelante (forward pass):**
        * Calcular la predicción de la calificación para cada usuario $j$ y película $i$:
            * $\hat{y}^{(i,j)} = w^{(j)} \cdot x^{(i)} + b^{(j)}$
    * **Cálculo de la pérdida:**
        * Calcular la función de pérdida, por ejemplo, el error cuadrático medio (MSE):
            * $L = \frac{1}{m} \sum_{i,j} (y^{(i,j)} - \hat{y}^{(i,j)})^2$
            * Donde $m$ es el número de calificaciones en el conjunto de entrenamiento.
    * **Cálculo de las derivadas (gradientes):**
        * Calcular las derivadas parciales de la pérdida respecto a cada parámetro:
            * $\frac{\partial L}{\partial w^{(j)}}$

            * $\frac{\partial L}{\partial x^{(i)}}$
            
            * $\frac{\partial L}{\partial b^{(j)}}$
    * **Actualización de parámetros:**
        * Actualizar los parámetros utilizando una tasa de aprendizaje $\alpha$:
            * $w^{(j)} = w^{(j)} - \alpha \frac{\partial L}{\partial w^{(j)}}$

            * $x^{(i)} = x^{(i)} - \alpha \frac{\partial L}{\partial x^{(i)}}$
            
            * $b^{(j)} = b^{(j)} - \alpha \frac{\partial L}{\partial b^{(j)}}$

**Convergencia:**

* El bucle de entrenamiento se detiene cuando la pérdida $L$ alcanza un nivel bajo o deja de disminuir significativamente.
* El descenso de gradiente iterativamente ajusta los parámetros del modelo para minimizar la función de pérdida. Se mueve en la dirección opuesta al gradiente, que representa la dirección de la pendiente más pronunciada (negativamente en un campo de n-dimensiones).
* La tasa de aprendizaje controla el tamaño del paso en cada iteración.
* La implementación específica de los cálculos de los gradientes puede variar dependiendo del del Problema o método de aprendizaje automático utilizado. 

@ Angel Alaguera



In [15]:
iterations = 200
lambda_ = 1

# Initialize random weights and biases
#np.random.seed(0)
#W = np.random.randn(num_movies, num_features)
#b = np.zeros(num_movies)

for iter in range(iterations):
    # Use GradientTape de TensorFlow
    # para registrar las operaciones utilizadas para calcular el costo 
    with tf.GradientTape() as tape:

        # Calcular el costo (pase hacia adelante incluido en el costo)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Cinta de gradiente para recuperar automáticamente los gradientes de las variables entrenables con respecto a la pérdida
    grads = tape.gradient( cost_value, [X,W,b] )

    # Aplicar paso de descenso de gradiente actualizando el valor de las variables para minimizar la pérdida.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Registrar la pérdida cada 20 iteraciones.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")
        #print(f"Época {epoch}, Iteración {iter}: Pérdida de Entrenamiento = {cost_value:0.2f}")

Training loss at iteration 0: 2321191.3
Training loss at iteration 20: 136168.7
Training loss at iteration 40: 51863.3
Training loss at iteration 60: 24598.8
Training loss at iteration 80: 13630.4
Training loss at iteration 100: 8487.6
Training loss at iteration 120: 5807.7
Training loss at iteration 140: 4311.6
Training loss at iteration 160: 3435.2
Training loss at iteration 180: 2902.1


## 6. Recomendaciones

* A continuación, se calcula las calificaciones de todas las películas y usuarios, mostramos las películas recomendadas. 

* Las películas y las calificaciones se recomiendad segun los datos ingresados en `my_ratings[]` que se Definio anteriormente. Para predecir la calificación de la película $i$ para el usuario $j$, calcula $\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)}$. Esto se puede calcular para todas las calificaciones utilizando la multiplicación de matrices.

* Las películas recomendadas son las que tienen una calificación más alta que las que ya han sido calificadas por el usuario.


## Predicción con Pesos y Sesgos Entrenados

El modelo de filtrado colaborativo, tras su entrenamiento, ha aprendido los mejores pesos (`w`) y sesgos (`b`) para minimizar el error en las predicciones de las calificaciones de películas.  Ahora, se puede usar estos pesos y sesgos entrenados para predecir cómo un usuario calificaría una película que aún no ha visto.

## Recordar el Objetivo del Modelo

El objetivo del modelo de filtrado colaborativo es encontrar los pesos y sesgos que minimicen el error entre las predicciones y las calificaciones reales. Al minimizar este error, el modelo se vuelve más preciso y ofrece resultados más confiables. 

El modelo ha aprendido a "entender" las preferencias de los usuarios y las características de las películas, y utiliza esta información para realizar predicciones más precisas. 


In [16]:
# Hacer una predicción usando pesos y sesgos entrenados por el Modelo
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

# restaurar la media (mean)
pm = p + Ymean

my_predictions = pm[:,0]

# Ordenar predicciones
ix = tf.argsort(my_predictions, direction='DESCENDING')

# Iterar sobre los índices de las películas
for i in range(17):
    # Obtener el índice de la película actual
    j = ix[i]
     # Verificar si la película no ha sido calificada por el usuario actual
    if j not in my_rated:
        # Imprimir la predicción de la calificación para la película
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}')

print('\n\nOriginal vs Predicted ratings:\n')

# Iterar sobre las calificaciones del usuario actual
for i in range(len(my_ratings)):
    # Verificar si el usuario ha calificado la película
    if my_ratings[i] > 0:
        # Imprimir la calificación original y la predicción para la película
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}')

Predicting rating 4.49 for movie My Sassy Girl (Yeopgijeogin geunyeo) (2001)
Predicting rating 4.48 for movie Martin Lawrence Live: Runteldat (2002)
Predicting rating 4.48 for movie Memento (2000)
Predicting rating 4.47 for movie Delirium (2014)
Predicting rating 4.47 for movie Laggies (2014)
Predicting rating 4.47 for movie One I Love, The (2014)
Predicting rating 4.46 for movie Particle Fever (2013)
Predicting rating 4.45 for movie Eichmann (2007)
Predicting rating 4.45 for movie Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Predicting rating 4.45 for movie Into the Abyss (2011)


Original vs Predicted ratings:

Original 5.0, Predicted 4.90 for Shrek (2001)
Original 5.0, Predicted 4.84 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Predicted 2.13 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Original 5.0, Predicted 4.88 for Harry Potter and the Chamber of Secrets (2002)
Original 5.0, Predic

In [17]:
# Pasamos Los Resultados a un DataFrame, para visualizar mejor los reultados.

filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

,pred,mean rating,number of ratings,title
1743,4.030965,4.252336,107,"Departed, The (2006)"
2112,3.985287,4.238255,149,"Dark Knight, The (2008)"
211,4.477792,4.122642,159,Memento (2000)
929,4.887053,4.118919,185,"Lord of the Rings: The Return of the King, The..."
2700,4.796530,4.109091,55,Toy Story 3 (2010)
653,4.357304,4.021277,188,"Lord of the Rings: The Two Towers, The (2002)"
1122,4.004469,4.006494,77,Shaun of the Dead (2004)
1841,3.980647,4.000000,61,Hot Fuzz (2007)
3083,4.084633,3.993421,76,"Dark Knight Rises, The (2012)"
2804,4.434171,3.989362,47,Harry Potter and the Deathly Hallows: Part 1 (...


## Mejorando las Predicciones

 **Nota: las predicciones iniciales pueden tener un rango limitado, especialmente si se consideran películas con pocas calificaciones o que son poco populares.** 

**Para mejorar las predicciones, puedes incorporar información adicional, como:**

* **Calificaciones promedio:** Puedes enfocarte en películas con calificaciones promedio altas. Esto puede ayudar a encontrar películas que son generalmente bien recibidas por la comunidad.
* **Número de calificaciones:** Priorizar películas con más de 20 calificaciones (o un número similar) puede ayudar a obtener predicciones más confiables, ya que se basan en un mayor número de opiniones.

**Utilizando Pandas para la Selección:**

Pandas es una biblioteca de Python que ofrece herramientas útiles para trabajar con datos tabulares.  Puedes utilizar Pandas para filtrar el conjunto de datos de películas y seleccionar aquellas que cumplen con los criterios mencionados:

* **Filtrado por calificación promedio:** Puedes usar la función `mean()` para calcular la calificación promedio de cada película y luego filtrar el DataFrame para seleccionar las películas con calificaciones promedio altas.
* **Filtrado por número de calificaciones:**  Puedes utilizar la función `value_counts()` para contar el número de calificaciones que recibió cada película y luego filtrar para seleccionar las que tengan más de 20 calificaciones.

**Integración, Ejemplo:**

1. **Importar Pandas:**
   ```python
   import pandas as pd
   ```
2. **Cargar el conjunto de datos:**
   ```python
   # conjunto de datos de películas 'movie_df' (Modo de ejemplo)
   movie_df = pd.read_csv('movie_data.csv')
   ```
3. **Filtrar el conjunto de datos:**
   ```python
   # Seleccionar películas con calificación promedio alta
   top_rated_movies = movie_df[movie_df['average_rating'] > 4]

   # Seleccionar películas con más de 20 calificaciones
   popular_movies = movie_df[movie_df['num_ratings'] > 20]
   ```
4. **Utilizar las películas filtradas para la predicción:**
   ```python
   # Realiza predicciones para las películas seleccionadas
   # ...
   ```

Cada selección de películas y proceso de predicción (Recomendar Peliculas) deben adaptarse a la probelmaticas especificas del Proyecto de estudio asi como tambien la naturaleza del conjunto de datos.


